In [1]:
import torch
device = torch.device('cuda')
device

device(type='cuda')

# 1.  데이터 준비

In [ ]:
pip install datasets

In [4]:
import json
data = []
with open('merged_v.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

In [6]:
import json

data = []
with open('merged_v.json', 'r') as f:
    for line in f:
        json_line = json.loads(line)
        for item in json_line:  # 리스트 get 불가
            final_translation = item.get('최종번역문')
            original_text = item.get('원문')
            data.append({'ko': original_text, 'de': final_translation})

In [7]:
data[0]

{'ko': '네, 아보카도 씨앗이 자라면 이렇게 돼요.',
 'de': 'Ja, wenn die Avocadosamen wachsen, ist das so.'}

In [8]:
import pandas as pd

df_val = pd.DataFrame(data, columns=['ko', 'de'])

In [9]:
df_val.head()

,ko,de
0,"네, 아보카도 씨앗이 자라면 이렇게 돼요.","Ja, wenn die Avocadosamen wachsen, ist das so."
1,자네는 이게 어느 나라 접시인지 아는가?,"Weißt du, aus welchem Land dieser Teller kommt?"
2,여러분 꽃게 구입하실 때 등딱지 끝부분을 보셔야 돼요.,"Wenn Sie Krabben kaufen, müssen Sie auf die En..."
3,한번 화면으로 만나보시죠.,Sehen wir es uns auf dem Bildschirm an.
4,"목살, 갈매기살입니다.",Es sind Nackenfleisch und Kronfleisch.


In [10]:
# train
data_t = []
with open('merged_t.json', 'r') as f:
    for line in f:
        json_line = json.loads(line)
        for item in json_line:  # 리스트 get 불가
            final_translation = item.get('최종번역문')
            original_text = item.get('원문')
            data_t.append({'ko': original_text, 'de': final_translation})

In [11]:
df_t = pd.DataFrame(data_t, columns=['ko', 'de'])

In [12]:
df_t.head()

,ko,de
0,요즘 마스크를 자주 쓰니까 피부 관리 중요할 것 같습니다.,"Ich finde es wichtig, meine Haut zu pflegen, d..."
1,소독은 따로 안 해도 되나요?,Muss es nicht desinfiziert werden?
2,낡은 사진을 통해서 정말 소중한 이야기를 들었습니다.,Wir haben eine sehr wertvolle Geschichte durch...
3,아기들 앞에서 뛰어놀면 아이들에게 너무 좋을 것 같아요.,"Es wäre toll für die Kinder, wenn sie hier vor..."
4,모든 식물들도 울림을 주면 영향을 받습니다.,Alle Pflanzen sind auch von Klingelungen beein...


In [13]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_t)

In [14]:
# dataset = dataset.train_test_split(test_size=0.1)

# train_dataset = dataset['train']
# test_dataset = dataset['test']

In [15]:
dataset

Dataset({
    features: ['ko', 'de'],
    num_rows: 81197
})

In [16]:
from datasets import Dataset

dataset_v = Dataset.from_pandas(df_val)

test dataset + train dataset 하나로 만들기

In [17]:
dataset

Dataset({
    features: ['ko', 'de'],
    num_rows: 81197
})

In [18]:
dataset_v

Dataset({
    features: ['ko', 'de'],
    num_rows: 10136
})

In [19]:
from datasets import DatasetDict

dataset_dict = DatasetDict({
    'train': dataset,
    'validation': dataset_v
})

In [20]:
dataset = dataset_dict

# 2. 데이터 처리

In [21]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-ko-de'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/838k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [22]:
def preprocess_function(examples):
    inputs = [ex for ex in examples['ko']]
    targets = [ex for ex in examples['de']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True,)

Map:   0%|          | 0/81197 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10136 [00:00<?, ? examples/s]

In [23]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['ko', 'de', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 81197
    })
    validation: Dataset({
        features: ['ko', 'de', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10136
    })
})

# 3. Trainer API로 모델 미세 조정하기

In [24]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
pip install sacrebleu

In [25]:
from datasets import load_metric

metric = load_metric("sacrebleu")

<ipython-input-25-a15d7a2cd8c7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [26]:
import numpy as np

def compute_metrics(eval_preds):
    # 예측값/실제레이블 분리
    preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우
    if isinstance(preds, tuple):
        preds = preds[0]

    # 사람이 읽을 수 있게 변환 (패딩 토큰, 시작 토큰, 종료 토큰 등을 제거)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # 패딩토큰이면(레이블 없는 토큰) token_id로 치환
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리 (공백 제거,  metric.compute 입력 형식 맞추기)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [27]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
pip install accelerate -U

In [28]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"my-Helsinki-NLP/opus-mt-ko-de",
    evaluation_strategy="no", # 시간 오래걸림..
    save_strategy="epoch", # 매 epoch마다 저장
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3, # 공간 이슈
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,

    push_to_hub=True, # 모델 업로드
)

In [29]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 4. 학습

In [30]:
trainer.train()

Step,Training Loss
500,2.134900
1000,1.954100
1500,1.873600
2000,1.793800
2500,1.747700
3000,1.731200
3500,1.678100
4000,1.672800
4500,1.642600
5000,1.624500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62140]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62140]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=15225, training_loss=1.4579534841954023, metrics={'train_runtime': 1779.7668, 'train_samples_per_second': 136.867, 'train_steps_per_second': 8.554, 'total_flos': 1741304716001280.0, 'train_loss': 1.4579534841954023, 'epoch': 3.0})

In [31]:
max_length = 128
trainer.evaluate(max_length=max_length)

{'eval_loss': 1.373229742050171,
 'eval_bleu': 26.63613655109713,
 'eval_runtime': 482.8009,
 'eval_samples_per_second': 20.994,
 'eval_steps_per_second': 0.657,
 'epoch': 3.0}

In [41]:
with open('채식주의자4(원본).txt', 'r', encoding='utf-8') as f:
    korean_data = f.readlines()

In [42]:
from transformers import pipeline
translator = pipeline('translation_ko_to_de', model='my-Helsinki-NLP/opus-mt-ko-de')
translated_data = [translator(text)[0]['translation_text'] for text in korean_data]

In [43]:
for ko, de in zip(korean_data[:10], translated_data[:10]):
    print(f'한국어: {ko.strip()}')
    print(f'독일어: {de}')
    print()

한국어: 그 꿈을 꾸기 전날 아침 난 얼어붙은 고기를 썰고 있었지. 당신이 화를내며 재촉했어.
독일어: Am Morgen, bevor ich diesen Traum hatte, schnitt ich gefrorenes Fleisch. Du hast mich verärgert und hast es eilig.

한국어: 제기랄, 그렇게 꾸물대고 있을 거야?
독일어: Oh mein Gott, wirst du so kneifen?

한국어: 알지, 당신이 서두를 때면 나는 정신을 못 차리지. 다른 사람이 된 것처럼 허둥대고, 그래서 오히려 일들이 뒤엉키지. 빨리, 더 빨리. 칼을 쥔 손이 바빠서 목덜미가 뜨거워졌어. 갑자기 도마가 앞으로 밀렸어. 손가락을 벤 것, 식칼의 이가 나간 건 그 찰나야.
독일어: Weißt du, wenn du in Eile bist, kann ich nicht klar denken. Es ist so, als wäre ich eine andere Person geworden, also sind die Dinge so, als wäre ich eine andere Person geworden. Beeile dich, schneller. Die Hände des Schwertes waren beschäftigt. Plötzlich wurde mein Nacken heiß. Ich habe meinen Finger nach vorne gedrückt. Ich habe meinen Finger nach vorne gedrückt und die Zähne des Messers sind herausgekommen.

한국어: 검지손가락을 들어올리자 붉은 핏방울 하나가 빠르게 피어나고 있었어. 둥글게, 더 둥글게. 손가락을 입속에 넣자 마음이 편안해졌어. 선홍빛의 색깔과 함께, 이상하게도 그 들큼한 맛이 나를 진정시키는 것 같았어.
독일어: Als ich den Zeigefinger hebt, blühte schnell ein roter Bluttr

In [44]:
import json

with open('veg_4_marian.json', 'w', encoding='utf-8') as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=4)